In [1]:
# Importing Libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

In [2]:
# Ignoring Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importing Dataset
df = pd.read_csv('data.csv')

In [4]:
# Dimensions of Dataset
df.shape

(50882, 14)

In [5]:
# Checking for Null Count
df.isnull().sum()

ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health_Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64

In [6]:
# Dropping Columns with high percentage of Null Values
df = df.drop(columns=['ID', 'Holding_Policy_Duration', 'Holding_Policy_Type'], axis=1)

In [7]:
# Checking for Null Count
df.isnull().sum()

City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Upper_Age                  0
Lower_Age                  0
Is_Spouse                  0
Health_Indicator       11691
Reco_Policy_Cat            0
Reco_Policy_Premium        0
Response                   0
dtype: int64

In [8]:
# Dropping Null Values
df.dropna(axis=0, how='any', inplace=True)

In [9]:
# Checking for Null Count after removing all Null containing Rows
df.isnull().sum()

City_Code              0
Region_Code            0
Accomodation_Type      0
Reco_Insurance_Type    0
Upper_Age              0
Lower_Age              0
Is_Spouse              0
Health_Indicator       0
Reco_Policy_Cat        0
Reco_Policy_Premium    0
Response               0
dtype: int64

In [10]:
# Dimensions of Dataset after removing all Null containing Rows
df.shape

(39191, 11)

In [11]:
# Data
df.head()

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health_Indicator,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,C3,3213,Rented,Individual,36,36,No,X1,22,11628.0,0
1,C5,1117,Owned,Joint,75,22,No,X2,22,30510.0,0
3,C24,4378,Owned,Joint,52,48,No,X1,19,17780.0,0
4,C8,2190,Rented,Individual,44,44,No,X2,16,10404.0,0
5,C9,1785,Rented,Individual,52,52,No,X2,22,15264.0,1


In [12]:
# Resetting Index
df.reset_index(drop=True, inplace=True)

In [13]:
# Data after Resetting Index
df.head()

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health_Indicator,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,C3,3213,Rented,Individual,36,36,No,X1,22,11628.0,0
1,C5,1117,Owned,Joint,75,22,No,X2,22,30510.0,0
2,C24,4378,Owned,Joint,52,48,No,X1,19,17780.0,0
3,C8,2190,Rented,Individual,44,44,No,X2,16,10404.0,0
4,C9,1785,Rented,Individual,52,52,No,X2,22,15264.0,1


In [14]:
# Dataset Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39191 entries, 0 to 39190
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_Code            39191 non-null  object 
 1   Region_Code          39191 non-null  int64  
 2   Accomodation_Type    39191 non-null  object 
 3   Reco_Insurance_Type  39191 non-null  object 
 4   Upper_Age            39191 non-null  int64  
 5   Lower_Age            39191 non-null  int64  
 6   Is_Spouse            39191 non-null  object 
 7   Health_Indicator     39191 non-null  object 
 8   Reco_Policy_Cat      39191 non-null  int64  
 9   Reco_Policy_Premium  39191 non-null  float64
 10  Response             39191 non-null  int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 3.3+ MB


## Label Encoding for Categorical Features
### Features to Target
- City_Code, Accomodation_Type, 
- Health_Indicator, Is_Spouse, 
- Reco_Insurance_Type

In [15]:
# Calculating Number of Categories in Each Column
cat_col_set = ['City_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse', 'Health_Indicator']
for col in cat_col_set:
    category_set = list(dict(df[col].value_counts()).keys())
    print(f'{col} : {category_set}\n')

City_Code : ['C1', 'C2', 'C3', 'C4', 'C9', 'C6', 'C7', 'C8', 'C10', 'C5', 'C15', 'C17', 'C16', 'C11', 'C13', 'C20', 'C19', 'C12', 'C18', 'C14', 'C21', 'C23', 'C24', 'C22', 'C26', 'C29', 'C25', 'C28', 'C33', 'C27', 'C32', 'C34', 'C30', 'C35', 'C36', 'C31']

Accomodation_Type : ['Owned', 'Rented']

Reco_Insurance_Type : ['Individual', 'Joint']

Is_Spouse : ['No', 'Yes']

Health_Indicator : ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']



In [16]:
# Setting Encoding Criteria
enc = {
    'City_Code': {f'C{i}': i for i in range(1, 37)},
    'Accomodation_Type': {'Owned': 0, 'Rented': 1},
    'Reco_Insurance_Type': {'Individual': 0, 'Joint': 1},
    'Is_Spouse': {'No': 0, 'Yes': 1},
    'Health_Indicator': {f'X{i}': i for i in range(1, 10)}
}

In [17]:
# Encoding Criteria Preview
enc

{'City_Code': {'C1': 1,
  'C2': 2,
  'C3': 3,
  'C4': 4,
  'C5': 5,
  'C6': 6,
  'C7': 7,
  'C8': 8,
  'C9': 9,
  'C10': 10,
  'C11': 11,
  'C12': 12,
  'C13': 13,
  'C14': 14,
  'C15': 15,
  'C16': 16,
  'C17': 17,
  'C18': 18,
  'C19': 19,
  'C20': 20,
  'C21': 21,
  'C22': 22,
  'C23': 23,
  'C24': 24,
  'C25': 25,
  'C26': 26,
  'C27': 27,
  'C28': 28,
  'C29': 29,
  'C30': 30,
  'C31': 31,
  'C32': 32,
  'C33': 33,
  'C34': 34,
  'C35': 35,
  'C36': 36},
 'Accomodation_Type': {'Owned': 0, 'Rented': 1},
 'Reco_Insurance_Type': {'Individual': 0, 'Joint': 1},
 'Is_Spouse': {'No': 0, 'Yes': 1},
 'Health_Indicator': {'X1': 1,
  'X2': 2,
  'X3': 3,
  'X4': 4,
  'X5': 5,
  'X6': 6,
  'X7': 7,
  'X8': 8,
  'X9': 9}}

In [18]:
# Encoding
df.replace(enc, inplace=True)

In [19]:
# Updated Dataset Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39191 entries, 0 to 39190
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_Code            39191 non-null  int64  
 1   Region_Code          39191 non-null  int64  
 2   Accomodation_Type    39191 non-null  int64  
 3   Reco_Insurance_Type  39191 non-null  int64  
 4   Upper_Age            39191 non-null  int64  
 5   Lower_Age            39191 non-null  int64  
 6   Is_Spouse            39191 non-null  int64  
 7   Health_Indicator     39191 non-null  int64  
 8   Reco_Policy_Cat      39191 non-null  int64  
 9   Reco_Policy_Premium  39191 non-null  float64
 10  Response             39191 non-null  int64  
dtypes: float64(1), int64(10)
memory usage: 3.3 MB


In [20]:
# Updated Data Set
df.head()

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health_Indicator,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,3,3213,1,0,36,36,0,1,22,11628.0,0
1,5,1117,0,1,75,22,0,2,22,30510.0,0
2,24,4378,0,1,52,48,0,1,19,17780.0,0
3,8,2190,1,0,44,44,0,2,16,10404.0,0
4,9,1785,1,0,52,52,0,2,22,15264.0,1


In [21]:
# Feature Set and Class Set
X = df.iloc[0:, :-1]
y = df.iloc[0:, 10:]

In [22]:
%%time
# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=-1)

Wall time: 0 ns


In [23]:
%%time

# Recursive Feature Selection
rank_set = {}
ranker = 1

for n in range(1, 6):
    rfe = RFE(rf_clf, n_features_to_select=n)
    rfe.fit(X, y)
    feature_set = {k: v for k, v in zip(df.columns, rfe.get_support())}
    for k in feature_set.keys():
        if feature_set[k]:
            if k not in rank_set.values():
                rank_set[ranker] = k
                ranker += 1

print(rank_set)

{1: 'Region_Code', 2: 'Reco_Policy_Premium', 3: 'Lower_Age', 4: 'Upper_Age', 5: 'City_Code'}
Wall time: 6min 18s
